In [4]:
#Importing various packages to install
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels as sm
from sklearn.model_selection import train_test_split

from sklearn import linear_model, datasets

In [4]:
# adding citibikes databse for mobi bikes in Vancouver
'''
accessing the city bikes database for mobibikes
'''
network_id = "mobibikes"
url = f"https://api.citybik.es/v2/networks/{network_id}"
response = requests.get(url)
data = response.json()

# Load station data into DataFrame
stations_df = pd.DataFrame(data['network']['stations'])

# Display relevant information
print(stations_df[['name', 'free_bikes', 'empty_slots', 'latitude', 'longitude']].head())

#add a new column to stations_df. Adding together free_bikes and empty_slots to a new column called total_bikes

stations_df['total_bikes'] = stations_df['free_bikes'] + stations_df['empty_slots']
stations_df

                    name  free_bikes  empty_slots   latitude   longitude
0       Chilco & Barclay           8           10  49.291909 -123.140713
1   St George & Broadway           2           12  49.262321 -123.093060
2  Britannia Parking Lot           2           12  49.275882 -123.071865
3        Morton & Denman           9           17  49.288030 -123.142135
4    Thornton & National           7            7  49.273777 -123.092723


,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra,total_bikes
0,00fa94ad698dc4a9e4d708d6fd32f294,Chilco & Barclay,49.291909,-123.140713,2024-12-16T18:33:35.643460Z,8,10,"{'uid': '0092', 'renting': True, 'returning': ...",18
1,012d3e06901cc222b1c2cf0a2ace3a29,St George & Broadway,49.262321,-123.093060,2024-12-16T18:33:35.708393Z,2,12,"{'uid': '0248', 'renting': True, 'returning': ...",14
2,029a505bd4422a1afd127987757f71a6,Britannia Parking Lot,49.275882,-123.071865,2024-12-16T18:33:35.697250Z,2,12,"{'uid': '0232', 'renting': True, 'returning': ...",14
3,0438114d2e6b96118de69bc9775bb21e,Morton & Denman,49.288030,-123.142135,2024-12-16T18:33:35.627272Z,9,17,"{'uid': '0031', 'renting': True, 'returning': ...",26
4,0459b7e93703980b853cd65a9dc60596,Thornton & National,49.273777,-123.092723,2024-12-16T18:33:35.697302Z,7,7,"{'uid': '0233', 'renting': True, 'returning': ...",14
...,...,...,...,...,...,...,...,...,...
253,fbb4c06b719596c21f91fd51ef6d4710,7th & Heather,49.265089,-123.119425,2024-12-16T18:33:35.631380Z,8,10,"{'uid': '0081', 'renting': True, 'returning': ...",18
254,fe0c57d04d6e682d284d501446095fc9,Union & Dunlevy,49.277595,-123.095830,2024-12-16T18:33:35.696167Z,10,6,"{'uid': '0212', 'renting': True, 'returning': ...",16
255,fedff7a263c182df94bda7307059cc52,Richards & Helmcken,49.277141,-123.122589,2024-12-16T18:33:35.654067Z,3,13,"{'uid': '0138', 'renting': True, 'returning': ...",16
256,fef69fb400210d861107a61db954d037,Keefer & Abbott,49.279821,-123.108020,2024-12-16T18:33:35.630112Z,25,3,"{'uid': '0053', 'renting': True, 'returning': ...",28


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice and Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [1]:
# Fetch the API key from environment variables
api_key = os.environ.get("api_key")
if not api_key:
    raise ValueError("API key not found in environment variables.")

# Base URL for Foursquare Places Search API
url = "https://api.foursquare.com/v3/places/search"

# Define the search categories for bars and restaurants
categories = "13026,13000"  # FourSquare Category IDs for Bars and Restaurants

# Create Request header
headers = {
    "Accept": "application/json",
    "Authorization": api_key
}

# Create an empty list to store results
nearby_places = []

# Iterate over each bike station
for index, row in stations_df.iterrows():
    station_name = row['name']
    station_lat = row['latitude']
    station_lon = row['longitude']

    # Set up search parameters for Foursquare
    params = {
        "query": "bars, restaurants",  # Query for bars and restaurants
        "ll": f"{station_lat},{station_lon}",  # Latitude and Longitude of the bike station
        "categories": categories,  # Categories for Bars and Restaurants
        "radius": 1000,  # Search radius in meters
        "limit": 50  # Number of results to return
    }

    try:
        # Make the GET request to the Foursquare
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the JSON response
        result = response.json()
        places = result.get('results', [])

        if not places: 
            print(f"No places found near {station_name}.") ##Print no places if there are no restaurants around a Bike Station 
        else:
            for place in places:
                place_name = place.get('name', 'No name available')
                place_lat = place.get('geocodes', {}).get('main', {}).get('latitude', None)
                place_lon = place.get('geocodes', {}).get('main', {}).get('longitude', None)
                rating = place.get('rating', 'No rating available')  # Parse Ratings 
                popularity = place.get('popularity', 'No popularity available')  # Parse Popularity
                price = place.get('price', 'No price available')  # Parse Price 

                # Add details to the list if latitude and longitude are available
                if place_lat is not None and place_lon is not None:
                    nearby_places.append({
                        "Bike Station": station_name,
                        "Place": place_name,
                        "Place Latitude": place_lat,
                        "Place Longitude": place_lon,
                        "Rating": rating,
                        "Popularity": popularity,
                        "Price": price
                    })

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while searching for places near {station_name}: {e}")

# Display the results
print(nearby_places_df)

Put your parsed results into a DataFrame

In [ ]:
nearby_places_df.to_csv('nearby_places_df.csv', index=False) #Creating a CSV with the same name

In [ ]:
YELP_KEY = "9Zwk8qRcxX_NdEqdibNuqMsToaKJHu6dGJ9KJ2SzvYGYJJMdJhJ9xZOaa6N2JHDmjkJUq5iFgtWQsjTaSb58Eol-tkkHPaSJppBRpYltuzUscxPtCMPPhAc_jbNXZ3Yx" 

##REMOVE BEFORE PUSHING

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice & Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
# Call the Yelp API key from environment variables
api_key = os.environ.get("YELP_KEY")
if not api_key:
    raise ValueError("API key not found in environment variables.")

# Yelp API Business Search endpoint
url = "https://api.yelp.com/v3/businesses/search"

# Create headers for the Yelp API request
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {api_key}"  # Yelp requires Bearer token for Authorization
}

# Assuming `stations_df` is your bike station DataFrame
bike_stations_df = stations_df.head()  # Using head() to limit API calls for testing purposes

# Create an empty list to store the results
yelp_places = []

# Iterate over each bike station
for index, row in bike_stations_df.iterrows():
    station_name = row['name'] #Look at each station_name
    station_lat = row['latitude'] #Look at the coordinates of each station to set 1000m radius
    station_lon = row['longitude']

    # Set up search parameters for Yelp
    params = {
        "term": "restaurants",  # Searching for restaurants
        "latitude": station_lat,  # Latitude of the bike station
        "longitude": station_lon,  # Longitude of the bike station
        "radius": 1000,  # Search radius in meters
        "limit": 50  # Limit Number of Results 
    }

    try:
        # Make the GET request to the Yelp API
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        # Parse the JSON response
        result = response.json()
        businesses = result.get('businesses', [])

        if not businesses:
            print(f"No restaurants found near {station_name}.")
        else:
            for business in businesses:
                place_name = business.get('name', 'No name available')
                place_lat = business.get('coordinates', {}).get('latitude', None)
                place_lon = business.get('coordinates', {}).get('longitude', None)
                rating = business.get('rating', 'No rating')
                review_count = business.get('review_count', 'No review count')

                if place_lat is not None and place_lon is not None:
                    yelp_places.append({
                        "Bike Station": station_name,
                        "Place": place_name,
                        "Place Latitude": place_lat,
                        "Place Longitude": place_lon,
                        "Rating": rating,
                        "Review Count": review_count
                    })

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while searching for places near {station_name}: {e}")


# Display or save the DataFrame
print(yelp_nearby_places_df.head())
# nearby_places_df.to_csv("restaurants_near_bike_stations_with_ratings.csv", index=False)


       Bike Station            Place  Place Latitude  Place Longitude  Rating  \
0  Chilco & Barclay           Kingyo       49.290739      -123.137050     4.4   
1  Chilco & Barclay             Nook       49.291755      -123.135297     4.2   
2  Chilco & Barclay  Guu with Garlic       49.290249      -123.133814     4.3   
3  Chilco & Barclay           Tavola       49.291816      -123.135719     4.4   
4  Chilco & Barclay            Jinmi       49.290812      -123.136070     4.5   

   Review Count  
0          1224  
1           331  
2          1098  
3           394  
4            25  


In [16]:
yelp_nearby_places_df #Printing DF to check results

,Bike Station,Place,Place Latitude,Place Longitude,Rating,Review Count,Price
0,Chilco & Barclay,Kingyo,49.290739,-123.137050,4.4,1224,$$
1,Chilco & Barclay,Nook,49.291755,-123.135297,4.2,331,$$
2,Chilco & Barclay,Tavola,49.291816,-123.135719,4.4,394,$$$
3,Chilco & Barclay,Guu with Garlic,49.290249,-123.133814,4.3,1098,$$
4,Chilco & Barclay,Noah's Cafe,49.288276,-123.140028,4.7,47,No price available
...,...,...,...,...,...,...,...
12863,8th & Prince Edward,Los Cuervos Taqueria Y Cantina,49.257476,-123.091044,3.8,192,$$
12864,8th & Prince Edward,Sal y Limón,49.256904,-123.089246,3.9,679,$$
12865,8th & Prince Edward,Sushi Yama,49.262973,-123.097188,3.3,253,$$
12866,8th & Prince Edward,La Cantina De Don Porfirio,49.263921,-123.101637,5.0,2,No price available


Put your parsed results into a DataFrame

In [17]:
#Save yelp_nearby_places_df as csv
yelp_nearby_places_df.to_csv('yelp_nearby_places_df.csv', index=False)

In [6]:
yelp_nearby_places_df = pd.read_csv('yelp_nearby_places_df.csv')
yelp_nearby_places_df

,Bike Station,Place,Place Latitude,Place Longitude,Rating,Review Count,Price
0,Chilco & Barclay,Kingyo,49.290739,-123.137050,4.4,1224,$$
1,Chilco & Barclay,Nook,49.291755,-123.135297,4.2,331,$$
2,Chilco & Barclay,Tavola,49.291816,-123.135719,4.4,394,$$$
3,Chilco & Barclay,Guu with Garlic,49.290249,-123.133814,4.3,1098,$$
4,Chilco & Barclay,Noah's Cafe,49.288276,-123.140028,4.7,47,No price available
...,...,...,...,...,...,...,...
12863,8th & Prince Edward,Los Cuervos Taqueria Y Cantina,49.257476,-123.091044,3.8,192,$$
12864,8th & Prince Edward,Sal y Limón,49.256904,-123.089246,3.9,679,$$
12865,8th & Prince Edward,Sushi Yama,49.262973,-123.097188,3.3,253,$$
12866,8th & Prince Edward,La Cantina De Don Porfirio,49.263921,-123.101637,5.0,2,No price available


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The API that provided the more complete data was Yelp. Foursquare's documentation is robust but in performing the search parameters given coordinates, it was missing large amounts of data, only being able to pull basic restaurant information. While Yelp still had many missing datapoints, it still had the more complete information with including price. Foursquare was unable to pull complete information given the same search parameters with missing Popularity, price and rating information, leading it to have to be discarded due to incomplete information as any analysis for them would be skewed. 

Get the top 10 restaurants according to their rating

In [15]:
#find top 10 restaurants by rating 

top_10 = (
    yelp_nearby_places_df
    .drop_duplicates(subset=['Place']) #Removes duplicates
    .sort_values(by=['Review Count', 'Rating'], ascending=[False, False])  # Sort by review count, then rating
    .head(10)  # Get the top 10 rows
) ##As there are many restaurant with similar ratings, sorting by review counts 

top_10

,Bike Station,Place,Place Latitude,Place Longitude,Rating,Review Count,Price
720,Robson & Hornby,Medina Cafe,49.280443,-123.117036,4.2,2741,$$
203,Thornton & National,Miku,49.286913,-123.113049,4.4,2428,$$$
200,Thornton & National,Phnom Penh,49.278431,-123.098205,4.2,1817,$$
222,Thornton & National,Chambar,49.280146,-123.109925,4.2,1630,$$
224,Thornton & National,Jam Cafe on Beatty,49.280259,-123.109638,4.4,1490,$$
300,Olympic Village Station,The Flying Pig,49.275010,-123.122000,4.0,1394,$$
707,Robson & Hornby,Joe Fortes Seafood & Chop House,49.284969,-123.124560,4.0,1371,$$$
0,Chilco & Barclay,Kingyo,49.290739,-123.137050,4.4,1224,$$
321,Olympic Village Station,Blue Water Cafe,49.276195,-123.120993,4.4,1155,$$$$
218,Thornton & National,Twisted Fork,49.283194,-123.104445,4.1,1147,$$
